# ANN FROM SCRATCH WITH 85% ACCURACY.

<img src='https://drive.google.com/uc?id=14nbP-PHEEtkaALbx0Yihr44qipu0Sz_l' width=1000 >

In [478]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [479]:
# Let us Import the Dataset.
df_train = pd.read_csv("input/train.csv")
df_test = pd.read_csv("input/test.csv")

In [480]:
# Top 5 Entries in Training Dataset
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [481]:
# This is to check the correlation between the Passenger Class and Survived ones.
df_train.groupby(['Pclass', 'Survived']).count()

PassengerId  Name  Sex    ...     Fare  Cabin  Embarked
Pclass Survived                            ...                          
1      0                  80    80   80    ...       80     59        80
       1                 136   136  136    ...      136    117       134
2      0                  97    97   97    ...       97      3        97
       1                  87    87   87    ...       87     13        87
3      0                 372   372  372    ...      372      6       372
       1                 119   119  119    ...      119      6       119

[6 rows x 10 columns]

* Upper class person Survived more then the other class person.

# BASIC EDA

In [482]:
# Let Us See the Basic Information about training dataset
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### 1). Let us get detail about Null Values.

In [483]:
null = df_train.isnull().sum()
per = null/len(df_train)*100
null = pd.DataFrame(data={'Number of Null Values':null, 'Percentage Of Null Values':per})
null

,Number of Null Values,Percentage Of Null Values
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


* As 77% of values in Cabin Column are Null values then we will drop the Cabin column.
* As 20% values in 'Age' column are Null values, here we will replace them by mean of the column.

### 2). Drop the Unwanted Columns.

In [484]:
df_train.drop(labels='Cabin', axis=1, inplace=True)
df_test.drop(labels='Cabin', axis=1, inplace=True)

### 3). Dealing With Missing Values.

* a). Deal with 'Age' column.

In [485]:
# Let's replace Age column with mean of the column.
print(df_train['Age'].mean())
df_train['Age'].fillna(value=df_train['Age'].mean(), inplace=True)
df_test['Age'].fillna(value=df_test["Age"].mean(), inplace=True)

29.69911764705882


* b). Deal with 'Embarked' column.

In [486]:
# Let us visualise the Embarked column.
df_train['Embarked'].value_counts().iplot(kind='bar', color='deepskyblue')

In [487]:
# Replace Embarked column with mode of the column which is 'S'.

df_train['Embarked'].fillna(value='S',inplace=True)
df_test['Embarked'].fillna(value='S', inplace=True)
df_train['Embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [488]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


# FEATURE ENGINEERING

### 1). Dealing With "Name" Column.
* Our Target is to replace the values of Name Column with Values 0,1,2,....


In [489]:
combine = [df_train, df_test]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df_train['Title'], df_train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [490]:

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
df_train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [491]:
# Mapping with 1,2,3,4,5
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


In [492]:
# Let us Drop the Unwanted Columns
df_train.drop(labels=['PassengerId','Name','Ticket'], inplace=True, axis=1)
df_test.drop(labels=['PassengerId','Name','Ticket'], inplace=True, axis=1)

### 2). Feature Engineering in "Age" Column.
* Here first we will see the band of Age and then we will replace values of Age in column which fall in particular band with 1 or 2 or 3 or 4.

In [493]:
df_train['AgeBand'] = pd.cut(df_train['Age'], 5)
df_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.344168
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [494]:
combine = [df_train, df_test]
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

df_train = df_train.drop(['AgeBand'], axis=1)
combine = [df_train, df_test]
df_train.head()    


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,1.0,1,0,7.2500,S,1
1,1,1,female,2.0,1,0,71.2833,C,3
2,1,3,female,1.0,0,0,7.9250,S,2
3,1,1,female,2.0,1,0,53.1000,S,3
4,0,3,male,2.0,0,0,8.0500,S,1


### 3). Dealing With "SibSp" and "Parch" columns.

In [495]:
combine = [df_train, df_test]
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [496]:
# Making New Column "IsAlone" which will have values 1 or 0 for whether she/he is alone or not respectively.
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

df_train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [497]:
# Droping Parch, SibSp and FamilySize columns.
df_train = df_train.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
df_test = df_test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [df_train, df_test]

df_train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,male,1.0,7.2500,S,1,0
1,1,1,female,2.0,71.2833,C,3,0
2,1,3,female,1.0,7.9250,S,2,1
3,1,1,female,2.0,53.1000,S,3,0
4,0,3,male,2.0,8.0500,S,1,1


### 4). Dealing With "FareBand" Column.

In [498]:
df_train['FareBand'] = pd.qcut(df_train['Fare'], 4)
df_train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [499]:
# Filling Null values in Fare column in test dataset.
df_test['Fare'].fillna(value=df_test['Fare'].mean(), inplace=True)

In [500]:
# Replacing values of Fare column with 0,1,2,3.

combine = [df_train, df_test]
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

df_train = df_train.drop(['FareBand'], axis=1)
combine = [df_train, df_test]
    
df_train.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,male,1.0,0,S,1,0
1,1,1,female,2.0,3,C,3,0
2,1,3,female,1.0,1,S,2,1
3,1,1,female,2.0,3,S,3,0
4,0,3,male,2.0,1,S,1,1
5,0,3,male,1.0,1,Q,1,1
6,0,1,male,3.0,3,S,1,1
7,0,3,male,0.0,2,S,4,0
8,1,3,female,1.0,1,S,3,0
9,1,2,female,0.0,2,C,3,0


## =============================================================

## PREDICTION WITH ANN
* 1). Here we will deal with MCSS.
    * M = dealing with Missing data, which we have already done.
    * C = dealing with Categorical values
    * S = Spliting of Dataset which we do not required as data is already splited.
    * S = Scaling of features on same scale.
* 2). Then we will create our model and then we will Train out model and then we will make prediction on our test dataset.

In [501]:
# Converting dataset into array.
X_train = df_train.iloc[:, 1:].values
y_train = df_train.iloc[:, 0].values
X_test = df_test.iloc[:,:].values

In [502]:
# Dealing with categorical features. 

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_x=LabelEncoder()
X_train[:, 1]=labelencoder_x.fit_transform(X_train[:,1])  
X_train[:, 4]=labelencoder_x.fit_transform(X_train[:, 4])

X_test[:, 1]=labelencoder_x.fit_transform(X_test[:,1])  
X_test[:, 4]=labelencoder_x.fit_transform(X_test[:, 4])


onehotencoder_x=OneHotEncoder(categorical_features=[0,2,3,4,5]) 
X_train=onehotencoder_x.fit_transform(X_train).toarray()
X_test=onehotencoder_x.fit_transform(X_test).toarray()


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning:

The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0,

In [503]:
len(X_train[0]), len(X_test[0])

(22, 22)

In [504]:
# Scaling the features on the same scale.
# Scaling of training set and test set must be done with the same sc_x object.
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.fit_transform(X_test)

In [505]:
# Importing tensorflow library
import tensorflow as tf
from tensorflow.keras.layers import Dropout

In [506]:
model = tf.keras.Sequential()

In [507]:
# Creating Input layer and 1st Hidden layer.
# Here Dropout is our regularization paramter(penalty parameter), which will deal with problem of overfitting .

model.add(tf.keras.layers.Dense(units=12, input_dim=22, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate=0.1))

In [508]:
# Creating 4 Hidden layers.

model.add(tf.keras.layers.Dense(units=12,  activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Dense(units=12,  activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Dense(units=12,  activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Dense(units=12,  activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate=0.1))

In [509]:
# Adding Output Layer.

model.add(tf.keras.layers.Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

In [510]:
# Compiling the Model.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [511]:
# Trian our Model.
history = model.fit(X_train, y_train, batch_size=50, epochs=500, validation_split=0.2)

Train on 712 samples, validate on 179 samples
Epoch 1/500
712/712 [==============================] - 1s 1ms/sample - loss: 0.6923 - acc: 0.6096 - val_loss: 0.6907 - val_acc: 0.6425
Epoch 2/500
712/712 [==============================] - 0s 141us/sample - loss: 0.6903 - acc: 0.6096 - val_loss: 0.6878 - val_acc: 0.6425
Epoch 3/500
712/712 [==============================] - 0s 149us/sample - loss: 0.6881 - acc: 0.6096 - val_loss: 0.6843 - val_acc: 0.6425
Epoch 4/500
712/712 [==============================] - 0s 149us/sample - loss: 0.6856 - acc: 0.6096 - val_loss: 0.6801 - val_acc: 0.6425
Epoch 5/500
712/712 [==============================] - 0s 144us/sample - loss: 0.6815 - acc: 0.6096 - val_loss: 0.6713 - val_acc: 0.6425
Epoch 6/500
712/712 [==============================] - 0s 144us/sample - loss: 0.6699 - acc: 0.6096 - val_loss: 0.6447 - val_acc: 0.6425
Epoch 7/500
712/712 [==============================] - 0s 145us/sample - loss: 0.6361 - acc: 0.6096 - val_loss: 0.5769 - val_acc: 0.64

712/712 [==============================] - 0s 151us/sample - loss: 0.3927 - acc: 0.8427 - val_loss: 0.3564 - val_acc: 0.8492
Epoch 120/500
712/712 [==============================] - 0s 143us/sample - loss: 0.3997 - acc: 0.8427 - val_loss: 0.3564 - val_acc: 0.8547
Epoch 121/500
712/712 [==============================] - 0s 145us/sample - loss: 0.3960 - acc: 0.8343 - val_loss: 0.3563 - val_acc: 0.8547
Epoch 122/500
712/712 [==============================] - 0s 140us/sample - loss: 0.4004 - acc: 0.8441 - val_loss: 0.3602 - val_acc: 0.8492
Epoch 123/500
712/712 [==============================] - 0s 142us/sample - loss: 0.4122 - acc: 0.8525 - val_loss: 0.3611 - val_acc: 0.8492
Epoch 124/500
712/712 [==============================] - 0s 141us/sample - loss: 0.3981 - acc: 0.8441 - val_loss: 0.3606 - val_acc: 0.8547
Epoch 125/500
712/712 [==============================] - 0s 142us/sample - loss: 0.3951 - acc: 0.8441 - val_loss: 0.3585 - val_acc: 0.8547
Epoch 126/500
712/712 [==================

712/712 [==============================] - 0s 144us/sample - loss: 0.3872 - acc: 0.8357 - val_loss: 0.3673 - val_acc: 0.8492
Epoch 179/500
712/712 [==============================] - 0s 145us/sample - loss: 0.3848 - acc: 0.8497 - val_loss: 0.3668 - val_acc: 0.8492
Epoch 180/500
712/712 [==============================] - 0s 142us/sample - loss: 0.3865 - acc: 0.8413 - val_loss: 0.3666 - val_acc: 0.8492
Epoch 181/500
712/712 [==============================] - 0s 152us/sample - loss: 0.3927 - acc: 0.8399 - val_loss: 0.3671 - val_acc: 0.8547
Epoch 182/500
712/712 [==============================] - 0s 146us/sample - loss: 0.4018 - acc: 0.8455 - val_loss: 0.3660 - val_acc: 0.8492
Epoch 183/500
712/712 [==============================] - 0s 141us/sample - loss: 0.4016 - acc: 0.8385 - val_loss: 0.3672 - val_acc: 0.8492
Epoch 184/500
712/712 [==============================] - 0s 144us/sample - loss: 0.3788 - acc: 0.8553 - val_loss: 0.3669 - val_acc: 0.8492
Epoch 185/500
712/712 [==================

712/712 [==============================] - 0s 149us/sample - loss: 0.3718 - acc: 0.8553 - val_loss: 0.3691 - val_acc: 0.8436
Epoch 238/500
712/712 [==============================] - 0s 149us/sample - loss: 0.3675 - acc: 0.8483 - val_loss: 0.3708 - val_acc: 0.8436
Epoch 239/500
712/712 [==============================] - 0s 154us/sample - loss: 0.3788 - acc: 0.8497 - val_loss: 0.3716 - val_acc: 0.8436
Epoch 240/500
712/712 [==============================] - 0s 148us/sample - loss: 0.3641 - acc: 0.8567 - val_loss: 0.3719 - val_acc: 0.8436
Epoch 241/500
712/712 [==============================] - 0s 150us/sample - loss: 0.3855 - acc: 0.8455 - val_loss: 0.3711 - val_acc: 0.8492
Epoch 242/500
712/712 [==============================] - 0s 146us/sample - loss: 0.3866 - acc: 0.8329 - val_loss: 0.3706 - val_acc: 0.8492
Epoch 243/500
712/712 [==============================] - 0s 149us/sample - loss: 0.3940 - acc: 0.8483 - val_loss: 0.3726 - val_acc: 0.8380
Epoch 244/500
712/712 [==================

712/712 [==============================] - 0s 144us/sample - loss: 0.3905 - acc: 0.8357 - val_loss: 0.3663 - val_acc: 0.8492
Epoch 297/500
712/712 [==============================] - 0s 150us/sample - loss: 0.3756 - acc: 0.8483 - val_loss: 0.3666 - val_acc: 0.8436
Epoch 298/500
712/712 [==============================] - 0s 141us/sample - loss: 0.3813 - acc: 0.8455 - val_loss: 0.3660 - val_acc: 0.8436
Epoch 299/500
712/712 [==============================] - 0s 143us/sample - loss: 0.3744 - acc: 0.8581 - val_loss: 0.3651 - val_acc: 0.8436
Epoch 300/500
712/712 [==============================] - 0s 137us/sample - loss: 0.3651 - acc: 0.8624 - val_loss: 0.3688 - val_acc: 0.8492
Epoch 301/500
712/712 [==============================] - 0s 141us/sample - loss: 0.3904 - acc: 0.8525 - val_loss: 0.3660 - val_acc: 0.8436
Epoch 302/500
712/712 [==============================] - 0s 155us/sample - loss: 0.3810 - acc: 0.8427 - val_loss: 0.3642 - val_acc: 0.8436
Epoch 303/500
712/712 [==================

712/712 [==============================] - 0s 143us/sample - loss: 0.3798 - acc: 0.8497 - val_loss: 0.3671 - val_acc: 0.8492
Epoch 356/500
712/712 [==============================] - 0s 137us/sample - loss: 0.3773 - acc: 0.8455 - val_loss: 0.3697 - val_acc: 0.8492
Epoch 357/500
712/712 [==============================] - 0s 143us/sample - loss: 0.3741 - acc: 0.8427 - val_loss: 0.3705 - val_acc: 0.8492
Epoch 358/500
712/712 [==============================] - 0s 148us/sample - loss: 0.3799 - acc: 0.8413 - val_loss: 0.3695 - val_acc: 0.8547
Epoch 359/500
712/712 [==============================] - 0s 145us/sample - loss: 0.3702 - acc: 0.8427 - val_loss: 0.3689 - val_acc: 0.8547
Epoch 360/500
712/712 [==============================] - 0s 143us/sample - loss: 0.3808 - acc: 0.8525 - val_loss: 0.3671 - val_acc: 0.8603
Epoch 361/500
712/712 [==============================] - 0s 143us/sample - loss: 0.3617 - acc: 0.8483 - val_loss: 0.3670 - val_acc: 0.8547
Epoch 362/500
712/712 [==================

712/712 [==============================] - 0s 143us/sample - loss: 0.3662 - acc: 0.8539 - val_loss: 0.3750 - val_acc: 0.8547
Epoch 415/500
712/712 [==============================] - 0s 146us/sample - loss: 0.3701 - acc: 0.8497 - val_loss: 0.3757 - val_acc: 0.8547
Epoch 416/500
712/712 [==============================] - 0s 145us/sample - loss: 0.3680 - acc: 0.8553 - val_loss: 0.3762 - val_acc: 0.8547
Epoch 417/500
712/712 [==============================] - 0s 141us/sample - loss: 0.3760 - acc: 0.8497 - val_loss: 0.3748 - val_acc: 0.8547
Epoch 418/500
712/712 [==============================] - 0s 144us/sample - loss: 0.3657 - acc: 0.8525 - val_loss: 0.3756 - val_acc: 0.8547
Epoch 419/500
712/712 [==============================] - 0s 140us/sample - loss: 0.3694 - acc: 0.8511 - val_loss: 0.3763 - val_acc: 0.8492
Epoch 420/500
712/712 [==============================] - 0s 145us/sample - loss: 0.3650 - acc: 0.8567 - val_loss: 0.3770 - val_acc: 0.8547
Epoch 421/500
712/712 [==================

712/712 [==============================] - 0s 146us/sample - loss: 0.3608 - acc: 0.8497 - val_loss: 0.3812 - val_acc: 0.8492
Epoch 474/500
712/712 [==============================] - 0s 142us/sample - loss: 0.3679 - acc: 0.8483 - val_loss: 0.3828 - val_acc: 0.8492
Epoch 475/500
712/712 [==============================] - 0s 146us/sample - loss: 0.3760 - acc: 0.8385 - val_loss: 0.3860 - val_acc: 0.8492
Epoch 476/500
712/712 [==============================] - 0s 142us/sample - loss: 0.3685 - acc: 0.8610 - val_loss: 0.3877 - val_acc: 0.8492
Epoch 477/500
712/712 [==============================] - 0s 139us/sample - loss: 0.3660 - acc: 0.8469 - val_loss: 0.3859 - val_acc: 0.8492
Epoch 478/500
712/712 [==============================] - 0s 146us/sample - loss: 0.3510 - acc: 0.8581 - val_loss: 0.3849 - val_acc: 0.8380
Epoch 479/500
712/712 [==============================] - 0s 155us/sample - loss: 0.3688 - acc: 0.8666 - val_loss: 0.3855 - val_acc: 0.8492
Epoch 480/500
712/712 [==================

In [512]:
print(f"Accuracy of training dataste\t {np.mean(history.history['acc'])}")
print(f"Accuracy of Validating dataset\t {np.mean(history.history['val_acc'])}")


Accuracy of training dataste	 0.8368426561355591
Accuracy of Validating dataset	 0.842257022857666


* Here we are getting the accuracy of 85%, which is good in my point of view.

In [513]:
# Prediction on Test dataset.
y_pred = model.predict(X_test)

In [514]:
# Converting the values of y_pred into 0 and 1.
# 1 means person will Survived and 0 means person will not Survived.

new_y_pred = []
for var in y_pred:
    if var>=0.7:
        new_y_pred.append(1)
    else:
        new_y_pred.append(0)

In [515]:

df_test = pd.read_csv("input/test.csv")
df_result = pd.DataFrame(data={'Passengerid':df_test['PassengerId'], 'Survived':new_y_pred})
df_result.head()

## ====================================================================